In [91]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

In [92]:
data = pd.read_json("cocktail_dataset.json")
data.set_index('id', inplace=True)

In [93]:
data.columns

Index(['name', 'category', 'glass', 'tags', 'instructions', 'imageUrl',
       'alcoholic', 'createdAt', 'updatedAt', 'ingredients'],
      dtype='object')

In [94]:
print(data.isnull().sum())

name             0
category         0
glass            0
tags            99
instructions     0
imageUrl         0
alcoholic        0
createdAt        0
updatedAt        0
ingredients      0
dtype: int64


In [95]:
data['tags'] = data['tags'].apply(lambda x: x if isinstance(x, list) else [])

Zaczynam od usunięcia imageUrl, createdAt, updatedAt, ponieważ nie wnoszą nic, a same daty dotyczą jednego dnia

In [96]:
data = data.drop(columns=["imageUrl", "createdAt", "updatedAt"])

In [97]:
# oficjalne IBA
with open("official_IBA.txt", "r", encoding="utf-8") as file:
    official_IBA = file.read().splitlines()

# tworzę nową kolumnę z wartościami 0 i 1 dla tagów IBA
data["IBA"] = 0
for i, row in data.iterrows():
    if row["name"] in official_IBA:
        data.at[i, "IBA"] = 1

In [98]:
data.head()

,name,category,glass,tags,instructions,alcoholic,ingredients,IBA
id,,,,,,,,
11000,Mojito,Cocktail,Highball glass,"[IBA, ContemporaryClassic, Alcoholic, USA, Asi...",Muddle mint leaves with sugar and lime juice. ...,1,"[{'id': 170, 'name': 'Soda water', 'descriptio...",1
11001,Old Fashioned,Cocktail,Old-fashioned glass,"[IBA, Classic, Alcoholic, Expensive, Savory]",Place sugar cube in old fashioned glass and sa...,1,"[{'id': 513, 'name': 'Water', 'description': '...",1
11002,Long Island Tea,Ordinary Drink,Highball glass,"[Strong, Asia, StrongFlavor, Brunch, Vegetaria...",Combine all ingredients (except cola) and pour...,1,"[{'id': 305, 'name': 'Light Rum', 'description...",0
11003,Negroni,Ordinary Drink,Old-fashioned glass,"[IBA, Classic]","Stir into glass over ice, garnish and serve.",1,"[{'id': 482, 'name': 'Sweet Vermouth', 'descri...",1
11004,Whiskey Sour,Ordinary Drink,Old-fashioned glass,"[IBA, Classic, Alcoholic, ContemporaryClassic]","Shake with ice. Strain into chilled glass, gar...",1,"[{'id': 409, 'name': 'Powdered Sugar', 'descri...",1


Dodaję każdemu wierszowi tag Alcoholic, zgodny z kolumną alcoholic

In [99]:
for i, row in data.iterrows():
    if type(row["tags"]) == list:
        if "Alcoholic" not in row["tags"]:
            row["tags"].append("Alcoholic")
    else:
        row["tags"] = ["Alcoholic"]

In [100]:
data.head()

,name,category,glass,tags,instructions,alcoholic,ingredients,IBA
id,,,,,,,,
11000,Mojito,Cocktail,Highball glass,"[IBA, ContemporaryClassic, Alcoholic, USA, Asi...",Muddle mint leaves with sugar and lime juice. ...,1,"[{'id': 170, 'name': 'Soda water', 'descriptio...",1
11001,Old Fashioned,Cocktail,Old-fashioned glass,"[IBA, Classic, Alcoholic, Expensive, Savory]",Place sugar cube in old fashioned glass and sa...,1,"[{'id': 513, 'name': 'Water', 'description': '...",1
11002,Long Island Tea,Ordinary Drink,Highball glass,"[Strong, Asia, StrongFlavor, Brunch, Vegetaria...",Combine all ingredients (except cola) and pour...,1,"[{'id': 305, 'name': 'Light Rum', 'description...",0
11003,Negroni,Ordinary Drink,Old-fashioned glass,"[IBA, Classic, Alcoholic]","Stir into glass over ice, garnish and serve.",1,"[{'id': 482, 'name': 'Sweet Vermouth', 'descri...",1
11004,Whiskey Sour,Ordinary Drink,Old-fashioned glass,"[IBA, Classic, Alcoholic, ContemporaryClassic]","Shake with ice. Strain into chilled glass, gar...",1,"[{'id': 409, 'name': 'Powdered Sugar', 'descri...",1


In [101]:
set(data.category)

{'Cocktail', 'Ordinary Drink', 'Punch / Party Drink'}

Są 3 kategorie drinków, więc zastosuję one-hot encoding

In [102]:
data = pd.get_dummies(data, columns=['category'])

dummy_columns = [col for col in data.columns if col.startswith('category_')]
data[dummy_columns] = data[dummy_columns].astype(int)

One-hot encoding zastosuję również na kolumnie glass

In [103]:
set(data.glass)

{'Brandy snifter',
 'Champagne flute',
 'Cocktail glass',
 'Collins glass',
 'Copper Mug',
 'Highball glass',
 'Old-fashioned glass',
 'Pousse cafe glass',
 'Whiskey Glass',
 'Whiskey sour glass',
 'White wine glass'}

In [104]:
data = pd.get_dummies(data, columns=['glass'])

dummy_columns = [col for col in data.columns if col.startswith('glass')]
data[dummy_columns] = data[dummy_columns].astype(int)

Na bazię kolumny name, dodam kolumnę ingredientInName. W EDA okazało się, że 25% drinków zawiera jakiś składnik w nazwię.

In [105]:
data["ingredientInName"] = 0

for i, row in data.iterrows():
    for ingredient in row["ingredients"]:
        if ingredient['name'] in row["name"]:
            data.at[i, "ingredientInName"] = 1

data.head()
data["ingredientInName"].value_counts()

ingredientInName
0    105
1     29
Name: count, dtype: int64

Teraz usuwam kolumne name, da się ją odzyskać po id.

In [106]:
data = data.drop(columns=["name"])

Dodaję kolumne instructionsLength określającą długość instrukcji w liczbie słów, dodatkowo skaluję tę kolumne używając StandardScaler().
Usuwam kolumne instructions.

In [107]:
data['instructions_length'] = data['instructions'].apply(lambda x: len(x.split()))

scaler = StandardScaler()
data['instructions_length'] = scaler.fit_transform(data[['instructions_length']])

data = data.drop(columns=["instructions"])

Dodaję kolejno ilość tagów tagsCount i ingredientsCount, po czym je skaluję.

In [112]:
data['tagsCount'] = data['tags'].apply(lambda x: len(x) if x is not None else 0)
data['ingredientsCount'] = data['ingredients'].apply(lambda x: len(x) if x is not None else 0)

data[['tagsCount', 'ingredientsCount']] = scaler.fit_transform(data[['tagsCount', 'ingredientsCount']])

In [115]:
data.head()

,tags,alcoholic,ingredients,IBA,category_Cocktail,category_Ordinary Drink,category_Punch / Party Drink,glass_Brandy snifter,glass_Champagne flute,glass_Cocktail glass,...,glass_Highball glass,glass_Old-fashioned glass,glass_Pousse cafe glass,glass_Whiskey Glass,glass_Whiskey sour glass,glass_White wine glass,ingredientInName,instructions_length,tagsCount,ingredientsCount
id,,,,,,,,,,,,,,,,,,,,,
11000,"[IBA, ContemporaryClassic, Alcoholic, USA, Asi...",1,"[{'id': 170, 'name': 'Soda water', 'descriptio...",1,1,0,0,0,0,0,...,1,0,0,0,0,0,0,1.278194,6.648649,0.819279
11001,"[IBA, Classic, Alcoholic, Expensive, Savory]",1,"[{'id': 513, 'name': 'Water', 'description': '...",1,1,0,0,0,0,0,...,0,1,0,0,0,0,0,1.600752,2.696036,0.029470
11002,"[Strong, Asia, StrongFlavor, Brunch, Vegetaria...",1,"[{'id': 305, 'name': 'Light Rum', 'description...",0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0.633079,4.277081,1.609087
11003,"[IBA, Classic, Alcoholic]",1,"[{'id': 482, 'name': 'Sweet Vermouth', 'descri...",1,0,1,0,0,0,0,...,0,1,0,0,0,0,0,-1.517304,1.114991,-0.760338
11004,"[IBA, Classic, Alcoholic, ContemporaryClassic]",1,"[{'id': 409, 'name': 'Powdered Sugar', 'descri...",1,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0.095483,1.905514,0.029470


Chociaż dałoby się wycisnąć z nich więcej, usuwam tagi i składniki.

In [ ]:
data = data.drop(columns=["tags", "ingredients"])